In [6]:
!pip3 install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.

In [7]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [14]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [24]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
# The params latitude and longitude are for NW France
params = {
    "latitude": 46.22,
    "longitude": 2.21,
    "hourly": ["temperature_2m", "dew_point_2m", "apparent_temperature", "precipitation_probability", "precipitation", "rain", "showers", "snowfall", "snow_depth"]
}
responses = openmeteo.weather_api(url, params=params)

In [25]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 46.220001220703125°E 2.2199997901916504°N
Elevation 391.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [26]:
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(2).ValuesAsNumpy()
hourly_precipitation_probability = hourly.Variables(3).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
hourly_rain = hourly.Variables(5).ValuesAsNumpy()
hourly_showers = hourly.Variables(6).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(7).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(8).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
    start = pd.to_datetime(hourly.Time(), unit = "s"),
    end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
    freq = pd.Timedelta(seconds = hourly.Interval()),
    inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["precipitation_probability"] = hourly_precipitation_probability
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["showers"] = hourly_showers
hourly_data["snowfall"] = hourly_snowfall
hourly_data["snow_depth"] = hourly_snow_depth

hourly_dataframe = pd.DataFrame(data = hourly_data)



In [27]:
hourly_dataframe

,date,temperature_2m,dew_point_2m,apparent_temperature,precipitation_probability,precipitation,rain,showers,snowfall,snow_depth
0,2023-11-28 00:00:00,7.559,5.698797,2.987986,100.0,0.1,0.0,0.1,0.0,0.0
1,2023-11-28 01:00:00,7.159,5.304710,2.372013,99.0,0.1,0.1,0.0,0.0,0.0
2,2023-11-28 02:00:00,7.109,5.091071,2.139391,98.0,0.3,0.2,0.0,0.0,0.0
3,2023-11-28 03:00:00,6.959,5.270201,2.172543,97.0,0.1,0.1,0.0,0.0,0.0
4,2023-11-28 04:00:00,6.509,4.500700,1.711694,93.0,0.7,0.6,0.1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
163,2023-12-04 19:00:00,0.373,-2.844382,-2.889452,19.0,0.2,0.2,0.0,0.0,0.0
164,2023-12-04 20:00:00,0.523,-2.698322,-2.771255,19.0,0.2,0.2,0.0,0.0,0.0
165,2023-12-04 21:00:00,0.623,-2.600950,-2.660353,19.0,0.2,0.2,0.0,0.0,0.0
166,2023-12-04 22:00:00,0.573,-2.145774,-2.545518,19.0,0.3,0.3,0.0,0.0,0.0
